## filtered_data_SEARO_1760085224294 to "month, year, dengue_total"

In [6]:
# Function to extract month
def substring_until_slash(s: str) -> str:
    """
    Return the middle substring
    """
    return s.split('-')[1]

In [7]:
import pandas as pd

# Load dataset
file_path = "filtered_data_SEARO_1760085224294.csv"
file = pd.read_csv(file_path)

# Select columns
columns_to_keep = ["calendar_start_date", "Year", "dengue_total"]
filtered_file = file[[col for col in columns_to_keep if col in file.columns]]

# Filter rows for year 2015–2024 only
filtered_file = filtered_file[filtered_file["Year"].between(2015, 2024, inclusive="both")]

# Get month and rename
filtered_file['calendar_start_date'] = filtered_file['calendar_start_date'].apply(substring_until_slash)
filtered_file.rename(columns={'calendar_start_date': 'month', 'Year': 'year'}, inplace=True)

# Filter by ascending order
sorted_file = filtered_file.sort_values(by=['year', 'month'], ascending=[True, True])

In [8]:
sorted_file.head()

,month,year,dengue_total
68,01,2015,21267
69,02,2015,20164
70,03,2015,14884
71,04,2015,13664
72,05,2015,11796


In [9]:
# Save filtered data to new CSV
output_path = "Indonesia_dengue_count_2015_2024.csv"
sorted_file.to_csv(output_path, index=False)

print(output_path)

Indonesia_dengue_count_2015_2024.csv


## Getting actual estimate population

In [2]:
import pandas as pd

df = pd.read_csv("Population per 1000 (Row Province, Column Year) 2018_2024.csv")

# Identify all numeric columns except the first one (e.g., 'Provinsi')
year_cols = df.columns.drop('Provinsi')

# Multiply those columns by 1000
df[year_cols] = df[year_cols] * 1000

df.to_csv("Population (Row Province, Column Year) 2018_2024.csv", index=False)

## Getting provincial annual dengue count
Provincial_annual_dengue_count = (Rate_of_dengue_incidence/100000) * Yearly_population_of_province

In [7]:
import pandas as pd

# Both datasets are sorted on province alphabetically. Both datasets have national count.
annual_rate_per_province = pd.read_csv("dengue_incidence 2015-2024 per 100000 cleaned.csv")
annual_population_per_province = pd.read_csv("Population (Row Province, Column Year) 2018_2024.csv")

# Find common years
common_years = sorted(set(annual_rate_per_province.columns) & set(annual_population_per_province.columns))

# Get annual_dengue_count. 
# Also round because there's no "half a person"
annual_dengue_count = annual_rate_per_province.copy()
annual_dengue_count[common_years] = round(
    (annual_rate_per_province[common_years] / 100000) *
    annual_population_per_province[common_years]
)

# Drop 2015, 2016, and 2017 cause it is not used
annual_dengue_count = annual_dengue_count.drop(["2015", "2016", "2017"], axis=1)

In [8]:
annual_dengue_count.head()

,Province,2018,2019,2020,2021,2022,2023,2024
0,Aceh,1522.0,2362.0,0.0,353.0,2006.0,2102.0,3380.0
1,Bali,908.0,5008.0,12107.0,2582.0,5537.0,6794.0,14933.0
2,Banten,1010.0,2867.0,2747.0,1940.0,4794.0,3764.0,12547.0
3,Bengkulu,1408.0,1464.0,1335.0,625.0,1335.0,926.0,3151.0
4,DI Yogyakarta,549.0,3323.0,3504.0,1097.0,1984.0,638.0,3669.0


In [10]:
annual_dengue_count.to_csv("Annual_dengue_count (Row Province, Column Year) 2018_2024.csv", index=False)

## Getting monthly weights (Indonesia total)
weight_per_month = dengue_that_month / dengue_that_year

In [20]:
import pandas as pd

df = pd.read_csv("Indonesia_dengue_count_2015_2024.csv")

# Sum all monthly counts by year
yearly_sum = df.groupby("year")["dengue_total"].transform("sum")

# Divide each month's count by the yearly total to get weights
df["monthly_weight"] = df["dengue_total"] / yearly_sum

In [23]:
# Drop rows where Year is in [2015, 2016, 2017, 2018, 2024]
df = df[~df["year"].isin([2015, 2016, 2017, 2018, 2024])]

In [24]:
print(df.head(20))

    month  year  dengue_total  monthly_weight
26      1  2019         27445        0.198688
27      2  2019         24018        0.173878
28      3  2019         18110        0.131107
29      4  2019         13642        0.098761
30      5  2019         12556        0.090899
31      6  2019          9514        0.068877
32      7  2019          7738        0.056019
33      8  2019          5386        0.038992
34      9  2019          4154        0.030073
35     10  2019          4376        0.031680
36     11  2019          5425        0.039274
37     12  2019          5767        0.041750
38      1  2020         15950        0.147272
39      2  2020         19693        0.181832
40      3  2020         19366        0.178813
41      4  2020         13160        0.121511
42      5  2020         10725        0.099028
43      6  2020          8106        0.074846
44      7  2020          6329        0.058438
45      8  2020          3833        0.035391


In [26]:
df.to_csv("Indonesia_dengue_weights 2019_2023.csv", index=False)

## Getting Jakarta monthly dengue count
Jakarta_dengue_count = Jakarta_yearly_dengue_count * that_month's_weight

In [31]:
import pandas as pd

annual = pd.read_csv("Annual_dengue_count (Row Province, Column Year) 2018_2024.csv")
weights = pd.read_csv("Indonesia_dengue_weights 2019_2023.csv")

# Select Jakarta's yearly data
jakarta = annual[annual['Province'].str.contains("Jakarta", case=False)].melt(
    id_vars=['Province'], var_name='year', value_name='annual_count'
)
jakarta['year'] = jakarta['year'].astype(int)

# Merge with Indonesia_dengue_weights by year
merged = pd.merge(weights, jakarta, on='year', how='inner')

# Compute Jakarta's monthly dengue count. Then round it.
merged['jakarta_monthly_count'] = round(merged['monthly_weight'] * merged['annual_count'])

# Clean output
jakarta_monthly = merged[['year', 'month', 'jakarta_monthly_count']].copy()

In [32]:
print(jakarta_monthly)

    year  month  jakarta_monthly_count
0   2019      1                 1721.0
1   2019      2                 1506.0
2   2019      3                 1136.0
3   2019      4                  855.0
4   2019      5                  787.0
5   2019      6                  597.0
6   2019      7                  485.0
7   2019      8                  338.0
8   2019      9                  260.0
9   2019     10                  274.0
10  2019     11                  340.0
11  2019     12                  362.0
12  2020      1                  702.0
13  2020      2                  866.0
14  2020      3                  852.0
15  2020      4                  579.0
16  2020      5                  472.0
17  2020      6                  357.0
18  2020      7                  278.0
19  2020      8                  169.0
20  2020      9                  121.0
21  2020     10                  106.0
22  2020     11                  130.0
23  2020     12                  133.0
24  2021      1          

In [33]:
jakarta_monthly.to_csv("Jakarta_dengue_count monthly 2019_2023.csv", index=False)